
$\qquad$ $\qquad$$\qquad$  **TDA231/DIT370 Discrete Optimization: Home Assignment 3 -- LP Duality and the Primal-Dual Algorithm** <br />
$\qquad$ $\qquad$$\qquad$                   **During grading time, direct queries regarding Q-1,2 to David & Q-3 to Adam** <br />
$\qquad$ $\qquad$$\qquad$                     **Due Date: 26/02/2023** <br />
$\qquad$ $\qquad$$\qquad$                   **Submitted by: Mirco Ghadri, 010421-1693, mircog@chalmers.se** <br />



---


General guidelines:
*   All solutions to theoretical and pratical problems must be submitted in this ipynb notebook and equations, wherever required, should be formatted using LaTeX math-mode.
*   All discussion regarding practical problems, along with solutions and plots should be specified in this notebook. All plots/results should be visible such that the notebook does not have to be run. But the code in the notebook should reproduce the plots/results if we choose to do so.
*   Your name, personal number and email address should be specified above.
*   All tables and other additional information should be included in this notebook.
*   Before submitting, make sure that your code can run on another computer. That all plots can show on another computer including all your writing. It is good to check if your code can run here: https://colab.research.google.com.


# Problem 1.

Consider the following LP problem:

\begin{alignat*}{2}
\max \ &4x_1-2x_2+5x_3+6x_4+7x_5\\
\\
\textrm{s.t} \quad
&2x_1 + 2x_2 - 4x_3 + 4x_4 + 8x_5 &&\leq 6\\
&2x_1 + \ \ {}x_2 - 2x_3 - \ \ x_4 - 3x_5 &&\geq -1\\
&5x_1 - 2x_2 + 4x_3 + 4x_4 + 2x_5 &&= 5\\
&2x_1 - 2x_2 + 5x_3 + 3x_4 + \ \ x_5 &&\leq 4\\
&\hspace{5.3cm} \vec x &&\geq \vec 0
\end{alignat*}

* (4 points) Write the LP dual of this problem.
* (3 points) Use CVXPY to compute the primal and dual optimum solutions and compare their values.
* (3 points) Check the complementary slackness conditions.

In [2]:
import numpy as np
import cvxpy as cp

### Method 1

There are 2 methods to solve this and formulate a dual to the LP. First we can express the LP in a standard form and then easily find the Dual to this LP, which will also be in a standard form. The standard form we want is.

\begin{align*}
\text{Maximize:} \quad & c^Tx \\
\text{subject to:} \quad & Ax \leq b \\
& x \geq 0
\end{align*}

In order to get this standard form, we can rewrite the equations using $\geq$ and $=$ to equations using $\leq$. For the case the equation uses $\geq$, we simply multiply the left and right hand side by -1 in order to get a corresponding equation with $\leq$. For the case the equation uses $=$, we simply replace it with 2 copies of the equation, one that uses $\geq$ instead, and one that uses $\leq$. Then, we replace the copy that uses $\geq$ to a version with $\leq$ by multiplying both sides by -1.

\begin{alignat*}{2}
\max \ &4x_1-2x_2+5x_3+6x_4+7x_5\\
\\
\textrm{s.t} \quad
&2x_1 + 2x_2 - 4x_3 + 4x_4 + 8x_5 &&\leq 6\\
&-2x_1 - x_2 + 2x_3 + x_4 + 3x_5 &&\leq 1\\
&5x_1 - 2x_2 + 4x_3 + 4x_4 + 2x_5 &&\leq 5\\
&-5x_1 + 2x_2 - 4x_3 - 4x_4 - 2x_5 &&\leq -5\\
&2x_1 - 2x_2 + 5x_3 + 3x_4 + x_5 &&\leq 4\\
&\hspace{5.3cm} \vec x &&\geq \vec 0
\end{alignat*}

Primal Problem **P**

In [5]:
X = cp.Variable(5, nonneg=True)
c = np.array([4,-2,5,6,7])
A = np.array([[2,2,-4,4,8],
              [-2,-1,2,1,3],
              [5,-2,4,4,2],
              [-5,2,-4,-4,-2],
              [2,-2,5,3,1]])
b = np.array([6,1,5,-5,4])
obj = cp.Maximize(c@X)
constraints = [A@X <= b]
problem = cp.Problem(obj,constraints)
result = problem.solve()

In [6]:
result

9.220338982968109

In [16]:
np.set_printoptions(suppress=True)
X.value

array([0.62711864, 2.81355932, 1.54237288, 0.        , 0.66101695])

Dual Problem **D**

In [100]:
Y = cp.Variable(5, nonneg=True)
c = np.array([4,-2,5,6,7])
A = np.array([[2,2,-4,4,8],
              [-2,-1,2,1,3],
              [5,-2,4,4,2],
              [-5,2,-4,-4,-2],
              [2,-2,5,3,1]])
b = np.array([6,1,5,-5,4])
obj = cp.Minimize(b@Y)
constraints = [A.T@Y >= c]
problem = cp.Problem(obj,constraints)
result = problem.solve()

In [101]:
result

9.220338972602596

As we can see, both the Primal problem **P** and the Dual problem **D** have the same optimal objective value. This follows from the Duality Theorem.

In [14]:
Y.value

array([0.52542373, 0.37288135, 0.8279979 , 0.48901485, 1.        ])

**c) Complementary slackness conditions**

The complementary slackness theorem allows us to find the optimal solution for the dual problem given that we know the optimal solution to the primal problem. We know that the optimal solution to the primal problem, $x^*=(0.62711864, 2.81355932, 1.54237288, 0.        , 0.66101695)$  
The complementary slackness theorem gives us 2 equations:  
$1.(b - A x_0) \cdot y_0 = 0 $  
$2.(A^T y_0 - c) \cdot x_0 = 0 $  
The first equation tells us that if the slackness in the primal constraint $i$ is greater than 0, than $y_i$ must be 0. The second equation tells us that constraint $i$ in the dual problem hold with equality if $x_i>0$.

So we start by calculating the slackness for all constraints in the primal LP.

In [18]:
(b-A@X).value

array([-0.,  0., -0.,  0.,  0.])

In [22]:
(A.T@Y-c).value

array([ 0.        ,  0.        , -0.        ,  0.83050847, -0.00000001])

We can see that the slackness in all constraints is equal to 0. This means $y_i > 0$ for all $i$ which is correct if we look at the optimal solution of the dual above.  We now look at the second equation. If we look at $x^*$, all elements are greater than 0 except for the 4th element. This means that constraints 1,2,3 and 5 will hold with equality in the dual problem which we also can confirm as show above. The dual problem looks like this:

\begin{align*}
\text{Minimize: }  b^Ty =  & \ 6y_1 + y_2 + 5y_3 - 5y_4 + 4y_5 \\
\text{Subject to: } & \\
&2y_1 - 2y_2 + 5y_3 - 5y_4 + 2y_5 \geq 4 \\
&2y_1 - y_2 - 2y_3 + 2y_4 - 2y_5 \geq -2 \\
&-4y_1 + 2y_2 + 4y_3 - 4y_4 + 5y_5 \geq 5 \\
&4y_1 + y_2 + 4y_3 - 4y_4 + 3y_5 \geq 6 \\
&8y_1 + 3y_2 + 2y_3 - 2y_4 + y_5 \geq 7 \\
&y_1, y_2, y_3, y_4, y_5 \geq 0
\end{align*}

Since constraints 1,2,3 and 5 hold with equality, we get:
\begin{align*}
2y_1 - 2y_2 + 5y_3 - 5y_4 + 2y_5 &= 4 \\
2y_1 - y_2 - 2y_3 + 2y_4 - 2y_5 &= -2 \\
-4y_1 + 2y_2 + 4y_3 - 4y_4 + 5y_5 &= 5 \\
8y_1 + 3y_2 + 2y_3 - 2y_4 + y_5 &= 7 \\
\end{align*}

This does not let us solve the values of y exactly since there are 4 equations and 5 variables. However, its the best we can do. The first condition in the CST allowed us to determine that all y must be greater than 0 since the slack of all equations in the primal problem was 0. The second condition in the CST told us that 4 equations must hold with equality since x has 5 variables and 4 were non-zero. This meants the slack of the corresponding equations in the dual had to be 0 so the equations needed to hold with equality. We can not solve these equations directly using guassian elimination since there are fewer equations than there are unknown variables. I found this link but It only mentions that finding a solution is hard: https://math.stackexchange.com/questions/4556920/complementary-slackness-condition-for-degenerate-cases

### Method 2

\begin{alignat*}{2}
\max \ &4x_1-2x_2+5x_3+6x_4+7x_5\\
\\
\textrm{s.t} \quad
&2x_1 + 2x_2 - 4x_3 + 4x_4 + 8x_5 &&\leq 6\\
&2x_1 + \ \ {}x_2 - 2x_3 - \ \ x_4 - 3x_5 &&\geq -1\\
&5x_1 - 2x_2 + 4x_3 + 4x_4 + 2x_5 &&= 5\\
&2x_1 - 2x_2 + 5x_3 + 3x_4 + \ \ x_5 &&\leq 4\\
&\hspace{5.3cm} \vec x &&\geq \vec 0
\end{alignat*}

LP Primal Problem

Here, we don't add the constraints on the form of $A x \leq b$. We add each constraint individually instead.

In [8]:
X = cp.Variable(5, nonneg=True)
c = np.array([4,-2,5,6,7])

A = np.array([[2,2,-4,4,8],
              [2,1,-2,-1,-3],
              [5,-2,4,4,2],
              [2,-2,5,3,1]])
b = np.array([6,-1,5,4])

constraints = [A[0]@X <= b[0], A[1]@X >= b[1], A[2]@X == b[2], A[3]@X <= b[3]]

obj = cp.Maximize(c@X)
problem = cp.Problem(obj,constraints)
result = problem.solve()

In [9]:
result

9.22033898268882

In [10]:
X.value

array([0.62711864, 2.81355932, 1.54237288, 0.        , 0.66101695])

For the dual, we use the dualization recipe presented in chapter 6.2 in the book

In [11]:
Y = cp.Variable(4)
c = np.array([4,-2,5,6,7])
A = np.array([[2,2,-4,4,8],
              [2,1,-2,-1,-3],
              [5,-2,4,4,2],
              [2,-2,5,3,1]])
b = np.array([6,-1,5,4])

A_t = A.T
constraints = [A_t[0]@Y >= c[0], A_t[1]@Y >= c[1], A_t[2]@Y >= c[2], A_t[3]@Y >= c[3], A_t[4]@Y >= c[4],
              Y[0]>=0, Y[1] <=0, Y[3] >= 0
              ]

obj = cp.Minimize(b@Y)
problem = cp.Problem(obj,constraints)
result = problem.solve()

In [12]:
result

9.220338969425676

# Problem 2.

Consider the LP problem:
\begin{alignat*}{2}
\max \ &6x_1 - 5x_3\\
\\
\textrm{s.t} \quad
&6x_1 - 3x_2 + x_3 &&= 2\\
&3x_1 + 4x_2 + x_3 &&\leq 5\\
&\ \ x_1 - 7x_2 &&\leq 5\\
&\hspace{2.45cm} x_1 &&\geq 0\\
&\hspace{2.45cm} x_2 &&\leq 0\\
&\hspace{2.45cm} x_3 &&\text{ unrestricted}
\end{alignat*}

* (3 points) Write the LP dual of this problem.
* (4 points) Consider the feasible solution $\vec x = (0,0,2)$ to the primal. Check if this solution is optimal by using the complementary slackness conditions to write down the corresponding dual solution.
* (3 points) Use complementary slackness to check if the primal feasible solution $\vec x = (1,0,-4)$ is optimal.

In [58]:
X = cp.Variable(3)
A = np.array([[6,-3,1],[3,4,1],[1,-7,0]])
cost = np.array([6,0,-5])
constraints = [X[0]>=0, X[1]<=0 ]
constraints.append((A[0]@X) == 2)
constraints.append((A[1]@X) <= 5)
constraints.append((A[2]@X) <= 5)
obj = cp.Maximize(cost@X)
problem = cp.Problem(obj,constraints)
result = problem.solve()
result

170.00000012893852

In [59]:
X.value

array([  5.        ,   0.        , -28.00000002])

We can use the dualization recipe presented in chapter **6.2** in the course book. We get that

$$
y_1 \in \mathbb{R}
$$

$$
y_2 \geq 0
$$

$$
y_3 \geq 0
$$



The constraints the Dual LP will be the transpose of the constraint matrix. The constraint matrix is:

In [46]:
A = np.array([[6,-3,1],[3,4,1],[1,-7,0]])
A.T

array([[ 6,  3,  1],
       [-3,  4, -7],
       [ 1,  1,  0]])

\begin{align*}
6y_1 + 3y_2 + y_3 &\geq 6 \\
-3y_1 + 4y_2 - 7y_3 &\leq 0 \\
y_1 + y_2 &= -5
\end{align*}

The objective function will $\mathbf{b}^T*y$ where b is $[2,5,5]$. So our final Dual LP can be expressed as:

\begin{align*}
\text{minimize} \quad & 2y_1 + 5y_2 + 5y_3 \\
\text{subject to} \quad & 6y_1 + 3y_2 + y_3 \geq 6 \\
& -3y_1 + 4y_2 - 7y_3 \leq 0 \\
& y_1 + y_2 = -5 \\
& y_1 \in \mathbb{R} \\
& y_2 \geq 0 \\
& y_3 \geq 0 \\
\end{align*}


In [63]:
Y = cp.Variable(3)
A = np.array([[6,3,1],[-3,4,-7],[1,1,0]])
cost = np.array([2,5,5])
constraints = [Y[1]>=0, Y[2]>=0 ]
constraints.append((A[0]@Y) >= 6)
constraints.append((A[1]@Y) <= 0)
constraints.append((A[2]@Y) == -5)
obj = cp.Minimize(cost@Y)
problem = cp.Problem(obj,constraints)
result = problem.solve()
result

170.00000026155527

In [64]:
Y.value

array([-4.99999992, -0.00000008, 36.0000001 ])

For reference, the primal was:

\begin{alignat*}{2}
\max \ &6x_1 - 5x_3\\
\\
\textrm{s.t} \quad
&6x_1 - 3x_2 + x_3 &&= 2\\
&3x_1 + 4x_2 + x_3 &&\leq 5\\
&\ \ x_1 - 7x_2 &&\leq 5\\
&\hspace{2.45cm} x_1 &&\geq 0\\
&\hspace{2.45cm} x_2 &&\leq 0\\
&\hspace{2.45cm} x_3 &&\text{ unrestricted}
\end{alignat*}

**b)** Consider the feasible solution $\vec x = (0,0,2)$ to the primal. Check if this solution is optimal by using the complementary slackness conditions to write down the corresponding dual solution.

When we use the complementary slackness conditions, we get 2 equations as mentioned earlier in question 1. The 2 equations are:  
$1.(b - A x_0) \cdot y_0 = 0 $  
$2.(A^T y_0 - c) \cdot x_0 = 0 $  
Equation 1 tells us that if the slack from constraint $i$ in the primal problem is greater than 0, then $y_i$ must be 0 in order for the equation to hold. We can calculate the slack of the constraints in the primal problem. The slack of the first constraint is 0 since it holds with equality. The slack of the second constraint is:  
$$
5 - (3x_1 + 4x_2 + x_3) = 5 - (3 \cdot 0 + 4 \cdot 0 + 2) = 5 - (0 + 0 + 2) = 5 - 2 = 3
$$  
The slack of the third constraint is:  
$$
5 - (x_1 - 7x_2) = 5 - (0 - 7 \cdot 0) = 5 - (0 - 0) = 5 - 0 = 5
$$  
Hence, we know that $y_2=0$ and $y_3=0$.  

Now, if we look at the second equation and what it tells us. It tells us that if $x_i$ is greater than 0, then slack from constraint $i$ in the Dual LP must be equal to 0. We know that $x_3=2$ so this means the slack from constraint 3 in the dual LP must be equal to 0. Since we know that $y_3$ is 0, we can rewrite the 3rd constraint in the dual LP as:  
$y_1 = -5$  
The slack is equal to $-5-y_1=0$ which means that $y_1=-5$.  
This means that $y=(-5,0,0)$  
To confirm this solution is optimal, we must check that $$ c^T x = b^T y $$ which we can confirm by plugging in the values for x in its primal objective function and plugging in the values for y in its dual objective function. We get that:  
$$
\begin{align*}
6x_1 - 5x_3 &= 2y_1 + 5y_2 + 5y_3 \\
6(0) - 5(2) &= 2(-5) + 5(0) + 5(0) \\
0 - 10 &= -10 + 0 + 0 \\
-10 &= -10
\end{align*}
$$  
However, there is one final step. We must check whether $y$ is a feasible solution. We do this by plugging in the values for y in the dual constraints. We get that y is not a feasible solution, since the constraints are not met. **Hence, x = (0,0,2) is not an optimal solution to the LP.**

**c)** Use complementary slackness to check if the primal feasible solution $\vec x = (1,0,-4)$ is optimal.

We repeat the same procedure described above. We start by calculating the slack for all constraints in the primal problem when $x=(1,0,-4)$. Since the first constraint holds with equality, the slack must be 0. Therefore we are only interested in the second and third constraint. 

$$
\begin{align*}
2. \quad & 5 - (3x_1 + 4x_2 + x_3) \\
&= 5 - (3(1) + 4(0) + (-4)) \\
&= 5 - (3 - 4) \\
&= 5 - (-1) \\
&= 6 \\
\\
3. \quad & 5 - (x_1 - 7x_2) \\
&= 5 - ((1) - 7(0)) \\
&= 5 - (1 - 0) \\
&= 5 - 1 \\
&= 4 \\
\end{align*}
$$

Since the slack for constraint 2 and 3 are greater than 0, this means that $y2=0$ and $y_3=0$. We now focus on the second equation in the CST. This tells us that if $x_i$ is not 0, then the slack from the dual constraint $i$ must be 0, so the constraint must therefore hold with equality. We have that $x_1$ and $x_3$ are not equal to 0. Hence we have:  
$$
\begin{align*}
& 6y_1 + 3y_2 + y_3 = 6 \\
& y_1 + y_2 = -5 \\
\end{align*}
$$  
Since $y_2$ and $y_3$ are equal to 0, this meanas that we can reformulate the equations as:
$$
\begin{align*}
& 6y_1 = 6 \\
& y_1 = -5 \\
\end{align*}
$$  
These equations do not have a solution, so this means that $\vec x = (1,0,-4)$ can not be an optimal solution. We can confirm that this is the case by solving the LP and finding the actual optimal value which is $\vec x = (5,0,-28)$

# Problem 3.

Consider the primal-dual algorithm for vertex cover discussed in class.
* (4 points) Run the algorithm by hand on the graph in the figure below (f*rom your previous homework). Show the values of the primal and dual variables at each iteration.
* (6 points) Implement the primal-dual algorithm as a python script to compute (approximate) vertex covers and run it for the random graph $G(100,0.1)$ from the previous homework.

<img src="vertex_cover.png" alt="Drawing" style="width: 180px;"/>

If the image does not load, try the direct link: https://tinyurl.com/tsnuz2c

<div>
    <img src="vertex_cover_step1.png" alt="Image 1" width="250px" float: left;">
    <img src="vertex_cover_step2.png" alt="Image 2" width="250px" float: left;">
    <img src="vertex_cover_step3.png" alt="Image 3" width="230px" float: left;">
</div>

From the image above, we can see the steps as we perform the primal dual algorithm on the graph to find a vertex cover. In total, we do 3 steps. In each step, we select an edge that is not yet frozen. We then increase its value y until one of its endpoints(nodes) becomes tight. We then freeze all edges incident to that node. We repeat until all edges are frozen. The tight nodes form the solution X and the frozen edges form the solution Y.

| Step |      x       |            y            |
|------|--------------|-------------------------|
|   1  | $x_4=1$      | $y_3=1, y_4=1, y_5=1, y_7=1$ |
|   2  | $x_4=1, x_1=1, x_2=1$ | $y_1=1, y_2=1, y_3=1, y_4=1, y_5=1, y_7=1$ |
|   3  | $x_4=1, x_1=1, x_2=1, x_5=1$ | $y_1=1, y_2=1, y_3=1, y_4=1, y_5=1, y_6=1, y_7=1$ |

**b)** Implement the primal-dual algorithm as a python script to compute (approximate) vertex covers and run it for the random graph $G(100,0.1)$ from the previous homework

In [54]:
import random

#this function is not working correctly. it does not remove all edges incident to a node
def vertex_cover(edges):
    frozen_edges = []
    tight_nodes = []
    remaining_edges = [e for e in edges if e not in frozen_edges]
    while remaining_edges:
        random_index = random.randint(0,len(remaining_edges)-1)
        random_edge = remaining_edges[random_index]
        #print("Random edge selected is",random_edge)
        v = random_edge[0]
        u = random_edge[1]
        #both nodes become tight since they have the same weight, which is 1.
        tight_nodes.append(v)
        tight_nodes.append(u)
        for e in remaining_edges:
            if (v in e) or (u in e):
                frozen_edges.append(e)
        remaining_edges = [edge for edge in edges if edge not in frozen_edges]
    return tight_nodes

In [55]:
adj_matrix = np.loadtxt("graph.txt")
edges = []
#add all of the edges to the edges array. Does not add a duplicate edge for example (u,v) and (v,u).
for u in range(len(adj_matrix)):
    for v in range(u,len(adj_matrix[0])):
        if adj_matrix[u][v]==1:
            edges.append((u,v))

In [72]:
len(vertex_cover(edges))

92

The size of the vertex cover becomes pretty large, approximately 90. This seems strange at first, since there are 100 vertices in the graph and we almost select all vertices to be in the vertex cover. However, when we remember the optimal solution calculated using Integer Programming, it was 70. Hence, this solution is not that bad and is within a factor 2 of the optimal solution, as it is supposed to be. We conclude by saying that the method works.